<a href="https://colab.research.google.com/github/MarioAvolio/AI-Project/blob/main/assignment%204/Rescal_with_PyKEEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install pykeen

In [2]:
import pykeen
pykeen.env()

Key,Value
OS,posix
Platform,Linux
Release,5.15.109+
Time,Tue Sep 5 17:58:39 2023
Python,3.10.12
PyKEEN,1.9.0
PyKEEN Hash,UNHASHED
PyKEEN Branch,
PyTorch,2.0.1+cu118
CUDA Available?,true


In [3]:
import pandas as pd
import numpy as np


### Load Training Data

In [4]:
data = pd.read_csv("https://raw.githubusercontent.com/ZhenfengLei/KGDatasets/master/Countries/Countries_S1/train.txt", sep="\t", names=["subject", "predicate", "object"])
data

,subject,predicate,object
0,western_africa,locatedin,africa
1,slovakia,neighbor,ukraine
2,slovakia,neighbor,hungary
3,slovakia,neighbor,austria
4,slovakia,neighbor,czechia
...,...,...,...
1106,cambodia,locatedin,south-eastern_asia
1107,panama,locatedin,central_america
1108,anguilla,locatedin,caribbean
1109,cuba,locatedin,caribbean


In [5]:
data_val = pd.read_csv("https://raw.githubusercontent.com/ZhenfengLei/KGDatasets/master/Countries/Countries_S1/valid.txt", sep="\t", names=["subject", "predicate", "object"])

In [6]:
entities = list(set(data["subject"].unique().tolist() + data["object"].unique().tolist()))

In [7]:
entities

['sweden',
 'french_guiana',
 'chile',
 'laos',
 'nicaragua',
 'australia',
 'syria',
 'senegal',
 'honduras',
 'hong_kong',
 'bangladesh',
 'papua_new_guinea',
 'western_asia',
 'svalbard_and_jan_mayen',
 'somalia',
 'saudi_arabia',
 'seychelles',
 'timor-leste',
 'greece',
 'africa',
 'uzbekistan',
 'fiji',
 'saint_vincent_and_the_grenadines',
 'micronesia',
 'philippines',
 'réunion',
 'north_korea',
 'denmark',
 'qatar',
 'macedonia',
 'belgium',
 'bermuda',
 'slovenia',
 'germany',
 'mauritania',
 'faroe_islands',
 'monaco',
 'singapore',
 'guernsey',
 'montenegro',
 'ireland',
 'zambia',
 'united_kingdom',
 'mexico',
 'south_sudan',
 'bahrain',
 'botswana',
 'northern_america',
 'vietnam',
 'bolivia',
 'el_salvador',
 'colombia',
 'swaziland',
 'gibraltar',
 'pakistan',
 'slovakia',
 'cocos_keeling_islands',
 'saint_kitts_and_nevis',
 'central_african_republic',
 'dominican_republic',
 'asia',
 'grenada',
 'isle_of_man',
 'iraq',
 'mongolia',
 'uruguay',
 'argentina',
 'united_st

In [8]:
number_of_entities = len(entities)

In [9]:
number_of_entities

271

## To PyKEEN TriplesFactory

In [10]:
from pykeen.triples import TriplesFactory

AttributeError: ignored

In [ ]:
data[["subject", "predicate", "object"]].values

In [ ]:
training_labels = TriplesFactory.from_labeled_triples(
  data[["subject", "predicate", "object"]].values,
  create_inverse_triples=False,
  entity_to_id=None,
  relation_to_id=None,
  compact_id=True,
  filter_out_candidate_inverse_relations=True,
  metadata=None,
)

testing_labels = TriplesFactory.from_labeled_triples(
  data_val[["subject", "predicate", "object"]].values,
  create_inverse_triples=False,
  entity_to_id=training_labels.entity_to_id,
  relation_to_id=training_labels.relation_to_id,
  compact_id=True,
  filter_out_candidate_inverse_relations=True,
  metadata=None,
)

## Training Pipeline

In [ ]:
from pykeen.pipeline import pipeline
import torch

In [ ]:
pipeline_result_rescal = pipeline(
    training=training_labels,
    testing=testing_labels,
    #dataset='Countries',
    model='Rescal',
    model_kwargs=dict(embedding_dim=80),
    random_seed=256,
    optimizer = torch.optim.Adam,
    optimizer_kwargs=dict(lr=1.0e-02),
    training_kwargs=dict(num_epochs=30),
)



In [ ]:
def print_metrics(pipeline_result):
  print(pipeline_result.training)
  print()
  print('hits@10', pipeline_result.get_metric('hits@10'))
  print('mr', pipeline_result.get_metric('mr'))
  print('mrr', pipeline_result.get_metric('mrr'))
  print()
  pipeline_result.plot_losses()

### Evaluation Metrics

In [ ]:
print_metrics(pipeline_result_rescal)

## Predict

In [ ]:
# canada + locatedin = ?
from pykeen.models import predict

predict.get_prediction_df(
    model=pipeline_result_rescal.model, head_label="canada", relation_label="locatedin", triples_factory=pipeline_result_rescal.training,
)

In [ ]:
countries_train_pd.query('head == "italy"')

In [ ]:
# italy + locatedin = ?
predict.get_prediction_df(
    model=pipeline_result_rescal.model, head_label="italy", relation_label="locatedin", triples_factory=pipeline_result_rescal.training,
)

In [ ]:
# italy + neighbor = ?
predict.get_prediction_df(
    model=pipeline_result_rescal.model, head_label="italy", relation_label="neighbor", triples_factory=pipeline_result_rescal.training,
).head(20)

In [ ]:
# vatican_city + neighbor = ?
predict.get_prediction_df(
    model=pipeline_result_rescal.model, head_label="vatican_city", relation_label="neighbor", triples_factory=pipeline_result_rescal.training,
)

## Visualize

In [ ]:
entities = list(countries.entity_to_id.keys())

In [ ]:
entity_embeddings = pipeline_result_rescal.model.entity_representations[0]().cpu().detach().numpy().astype('float32')
entity_embeddings.shape

In [ ]:
entity_embeddings

In [ ]:
relation_embeddings = pipeline_result_rescal.model.relation_representations[0]().cpu().detach().numpy().astype('float32')
relation_embeddings.shape

In [ ]:
entity_embeddings

In [ ]:
from operator import sub
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def quick_plot(labels, embedding, colors = None, plot = True):
  """
  we ignore the words we are not interested to reduce the time to compute
  the 2D embeddings, but this is an approximation.
  """

  if not colors:
    colors = ['b' for _ in labels]

  X = embedding
  pca = PCA(n_components=2)

  two_d_emb = pca.fit_transform(X)

  plt.scatter(two_d_emb[:, 0], two_d_emb[:, 1], c = colors)

  for index, label in enumerate(labels):
    # annotate
    plt.annotate(label, xy=(two_d_emb[index, 0], two_d_emb[index, 1]), )

  if plot:
    plt.gcf().set_size_inches(20, 10)
    plt.show()

  return plt, pca

In [ ]:
continents = set(countries_train_pd.query('relation == "locatedin"')['tail'].unique())

In [ ]:
def get_color(x):
  if x in continents:
    return 'red'
  else:
    return 'blue'

colors = [get_color(ent) for ent in entities]

In [ ]:
quick_plot(entities, entity_embeddings, colors=colors)